In [29]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [31]:
data=pd.read_csv("C:/Users/aditya/Desktop/2024/HireBolt.AI/Appliciants.csv",encoding='utf8')

In [32]:
data.head()

,Unnamed: 0,email_id,phone_number,summary
0,0,['gbp.bhaskar@gmail.com'],['8056248707'],"[[""Bhaskar G is an IT professional with 13 yea..."
1,1,[None],[None],[['The document is a list of references for va...
2,2,['amrit12june90@gmail.com'],['(+91)\n814\n351\n4822'],[['The document is a list of profiles from sev...
3,3,['adityabhatt19058568031.stats@rla.du.ac.in'],['+91 7 303041453'],[['Aditya Bhatt is a student with a PG Diploma...
4,4,['richabudhraja8@gmail.com'],['9090302071'],[['Richa Kaur is a business analyst with a bac...


In [33]:
data.drop("Unnamed: 0",axis=1,inplace=True)

In [34]:
data.head()

,email_id,phone_number,summary
0,['gbp.bhaskar@gmail.com'],['8056248707'],"[[""Bhaskar G is an IT professional with 13 yea..."
1,[None],[None],[['The document is a list of references for va...
2,['amrit12june90@gmail.com'],['(+91)\n814\n351\n4822'],[['The document is a list of profiles from sev...
3,['adityabhatt19058568031.stats@rla.du.ac.in'],['+91 7 303041453'],[['Aditya Bhatt is a student with a PG Diploma...
4,['richabudhraja8@gmail.com'],['9090302071'],[['Richa Kaur is a business analyst with a bac...


In [35]:
data['summary'] = data['summary'].apply(lambda x: x[3:-3] if type(x) is str and len(x) >= 6 else x)

In [36]:
data.head()

,email_id,phone_number,summary
0,['gbp.bhaskar@gmail.com'],['8056248707'],Bhaskar G is an IT professional with 13 years ...
1,[None],[None],The document is a list of references for vario...
2,['amrit12june90@gmail.com'],['(+91)\n814\n351\n4822'],The document is a list of profiles from severa...
3,['adityabhatt19058568031.stats@rla.du.ac.in'],['+91 7 303041453'],Aditya Bhatt is a student with a PG Diploma in...
4,['richabudhraja8@gmail.com'],['9090302071'],Richa Kaur is a business analyst with a backgr...


In [39]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text

# Apply the cleaning function to the 'summary' column
data['cleaned_summary'] = data['summary'].apply(clean_text)

In [46]:

data=data[data["email_id"] != "[None]"]
data.head()

,email_id,phone_number,summary,cleaned_summary,similarity_score
0,['gbp.bhaskar@gmail.com'],['8056248707'],Bhaskar G is an IT professional with 13 years ...,bhaskar g is an it professional with years of...,0.076268
2,['amrit12june90@gmail.com'],['(+91)\n814\n351\n4822'],The document is a list of profiles from severa...,the document is a list of profiles from severa...,0.042207
3,['adityabhatt19058568031.stats@rla.du.ac.in'],['+91 7 303041453'],Aditya Bhatt is a student with a PG Diploma in...,aditya bhatt is a student with a pg diploma in...,0.177969
4,['richabudhraja8@gmail.com'],['9090302071'],Richa Kaur is a business analyst with a backgr...,richa kaur is a business analyst with a backgr...,0.109644
5,['anandhuprakash184@gmail.com'],['8157965047'],Anandhu Prakash is a professional with experie...,anandhu prakash is a professional with experie...,0.103380


In [50]:
df=data
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['cleaned_summary'])

# Function to calculate cosine similarity
def calculate_similarity(query_vector, document_matrix):
    cosine_similarities = cosine_similarity(query_vector, document_matrix)
    return cosine_similarities.flatten()

# Define your job description
job_description = "Advanced knowledge of Tableau and/or Alteryx,Working knowledge either in Python or R is required ,Exceptional analytic and technical skills, including knowledge of market research tools, data science and business intelligence tools, ,Experience explaining and performing a variety of statistical and data mining techniques,Experience developing audience-right high quality presentations ,Exceptional interpersonal and communication skills,Proven self-starter (comfortable with what could be a remote team) with demonstrated problem solving and organizational skills"
cleaned_job_description = clean_text(job_description)
job_description_vector = vectorizer.transform([cleaned_job_description])

# Calculate cosine similarity with the job description
df['similarity_score'] = calculate_similarity(job_description_vector, tfidf_matrix)

# Sort the dataframe based on similarity scores
df_sorted = df.sort_values(by='similarity_score', ascending=False)

df_sorted.head().to_csv("selected appliciants.csv")